In [ ]:
# Install dependencies
import pandas as pd
import glob
import os

import matplotlib.pyplot as plt
import seaborn as sns
import random
import math

In [ ]:
# Load all parquet files function
def combine_parquet_files(directory):
    # Get all parquet files in the directory
    parquet_files = glob.glob(os.path.join(directory, '*.parquet'))
    
    # Read and concatenate all parquet files
    df_list = []
    for file in parquet_files:
        df = pd.read_parquet(file)
        df_list.append(df)
    
    # Combine all dataframes
    combined_df = pd.concat(df_list, ignore_index=True)
    
    return combined_df

In [ ]:
# Load the files
directory = 'data/data_parquet'
result_df = combine_parquet_files(directory)

result_df.head()

In [ ]:
# Some information
individual_accounts = result_df['name'].nunique()
ultimate_parents = result_df['ultimate_parent_id'].nunique()
print(f"There are {individual_accounts} individual accounts and {ultimate_parents} ultimate parents.") 

In [ ]:
# Aggregate the timeseries by ultimate parent
result_df['date'] = pd.to_datetime(result_df['date'])

# Group by 'group_id' and 'date', then sum the 'c' values
aggregated_df = result_df.groupby(['ultimate_parent_id', 'date'])['balance'].sum().reset_index()

# If you want to sort the result
aggregated_df = aggregated_df.sort_values(['ultimate_parent_id', 'date'])

aggregated_df.head()

In [ ]:
# Function to view the timeseries
def plot_random_timeseries_grid(df, n_series=10):
    # Set the style for the plot
    sns.set_style("whitegrid")
    plt.rcParams['font.size'] = 8

    # Get unique group_ids and randomly sample n_series of them
    unique_groups = df['ultimate_parent_id'].unique()
    if len(unique_groups) < n_series:
        print(f"Warning: Only {len(unique_groups)} unique groups available. Plotting all of them.")
        n_series = len(unique_groups)
    random_groups = random.sample(list(unique_groups), n_series)

    # Calculate grid dimensions
    n_cols = min(3, n_series)
    n_rows = math.ceil(n_series / n_cols)

    # Create subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 3*n_rows))
    fig.suptitle('Random Timeseries', fontsize=16, y=1.02)

    # Flatten axes array for easier indexing
    axes = axes.flatten() if n_series > 1 else [axes]

    # Plot each timeseries
    for i, group in enumerate(random_groups):
        group_data = df[df['ultimate_parent_id'] == group].sort_values('date')
        
        axes[i].plot(group_data['date'], group_data['balance'], linewidth=2)
        axes[i].set_title(f'Group ID: {group}')
        axes[i].tick_params(axis='x', rotation=45)
        axes[i].grid(True, linestyle='--', alpha=0.7)

    # Remove unused subplots
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])

    # Improve the layout
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
plot_random_timeseries_grid(aggregated_df, n_series=12)

In [ ]:
def filter_timeseries_by_delta(df):
    # Print the number of entries before filtering
    print(f"Number of entries before filtering: {len(df)}")
    print(f"Number of unique groups before filtering: {df['ultimate_parent_id'].nunique()}")

    # Function to calculate the proportion of non-zero deltas and check last 30 deltas
    def analyze_deltas(group):
        # Sort the group by date
        group = group.sort_values('date')
        # Calculate deltas
        deltas = group['balance'].diff()
        # Calculate proportion of non-zero deltas (excluding the first NaN)
        non_zero_delta_prop = (deltas[1:] != 0).mean()
        # Check if last 30 deltas are all zero
        last_30_all_zero = (deltas.tail(10) == 0).all()
        return pd.Series({'non_zero_delta_prop': non_zero_delta_prop, 'last_30_all_zero': last_30_all_zero})

    # Group by 'group_id' and apply the analysis
    group_analysis = df.groupby('ultimate_parent_id').apply(analyze_deltas)

    # Get the groups that meet both criteria:
    # 1. More than 50% of deltas are non-zero
    # 2. Last 30 deltas are not all zero
    valid_groups = group_analysis[
        (group_analysis['non_zero_delta_prop'] > 0.5) & 
        (~group_analysis['last_30_all_zero'])
    ].index

    # Filter the dataframe to keep only the valid groups
    df_filtered = df[df['ultimate_parent_id'].isin(valid_groups)]

    # Print the number of entries after filtering
    print(f"\nNumber of entries after filtering: {len(df_filtered)}")
    print(f"Number of unique groups after filtering: {df_filtered['ultimate_parent_id'].nunique()}")

    # Calculate and print the percentage of data removed
    percent_removed = (1 - len(df_filtered) / len(df)) * 100
    print(f"\nPercentage of data removed: {percent_removed:.2f}%")

    # Print reasons for removal
    removed_due_to_delta = group_analysis[group_analysis['non_zero_delta_prop'] <= 0.5].index
    removed_due_to_last_30 = group_analysis[group_analysis['last_30_all_zero']].index
    print(f"\nGroups removed due to <= 50% non-zero deltas: {len(removed_due_to_delta)}")
    print(f"Groups removed due to last 30 deltas being zero: {len(removed_due_to_last_30)}")

    return df_filtered

In [ ]:
df_filtered = filter_timeseries_by_delta(aggregated_df)

plot_random_timeseries_grid(df_filtered, n_series=10)

In [ ]:
df_filtered

In [ ]:
def pivot_anonymize_and_save(df, output_file):
    # Step 1: Create a mapping of original group_ids to anonymized ids
    unique_groups = df['ultimate_parent_id'].unique()
    group_mapping = {group: i for i, group in enumerate(unique_groups)}
    
    # Step 2: Apply the mapping to create a new anonymized group_id column
    df['index'] = df['ultimate_parent_id'].map(group_mapping)
    
    # Step 3: Pivot the dataframe
    pivoted_df = df.pivot(index='date', columns='index', values='balance')
    
    # Step 4: Reset the index to make 'date' a column again
    pivoted_df.reset_index(inplace=True)
    
    # Step 5: Sort the dataframe by date
    pivoted_df.sort_values('date', inplace=True)
    
    # Step 6: Save the pivoted and anonymized dataframe to a CSV file
    pivoted_df.to_csv(output_file, index=False)
    
    print(f"Pivoted and anonymized data saved to {output_file}")
    print(f"Number of unique group_ids: {len(unique_groups)}")
    print(f"Shape of pivoted dataframe: {pivoted_df.shape}")
    
    return pivoted_df

In [ ]:
# Pivot and save the data
pivoted_df = pivot_anonymize_and_save(df_filtered, 'data/eod_balances_2110_full.csv')

In [ ]:
pivoted_df

In [ ]:
# Function to remove columns and create tiny test dataframe
def remove_last_n_columns(df, n):
    # Check if n is greater than the number of columns
    if n >= len(df.columns):
        raise ValueError(f"n ({n}) must be less than the number of columns ({len(df.columns)})")
    
    # Calculate the number of columns to keep
    columns_to_keep = len(df.columns) - n
    
    # Create a new DataFrame with only the kept columns
    df_trimmed = df.iloc[:, :columns_to_keep]
    
    print(f"Removed last {n} columns")
    print(f"Original shape: {df.shape}")
    print(f"New shape: {df_trimmed.shape}")
    
    return df_trimmed

In [ ]:
df_trimmed = remove_last_n_columns(pivoted_df, 252)

df_trimmed

In [ ]:
# Save the small version too
df_trimmed.to_csv('data/eod_balances_2110_tiny.csv', index=False)